In [2]:
import eotdl


In this notebook we generate the dataset for the use case

1. Generate list of Satellogic images to be used (containing bb and acquisition time)
2. Explore available S1/S2 images with different criteria
	- bounding box overlap
	- acquisition time overlap
3. Download matching S1/S2 at given resolution
4. Generate metadata and ingest to EOTDL 

In [3]:
import geopandas as gpd

# gdf = gpd.read_parquet('/fastdata/Satellogic/data/satellogic-earthview-items.parquet')
gdf = gpd.read_parquet('~/DeLocal_Data/satellogic-earthview-items-with-matches.parquet')
# json_path, zone, region, date, geometry, matches
# json_path, zone, region, date, geometry
gdf.head()


FileNotFoundError: [Errno 2] Failed to open local file '/Users/benastahl/DeLocal_Data/satellogic-earthview-items-with-matches.parquet'. Detail: [errno 2] No such file or directory

In [4]:
gdf.shape

NameError: name 'gdf' is not defined

In [5]:
gdf['num_matches'] = gdf.matches.apply(len)
gdf['num_matches'].value_counts()

NameError: name 'gdf' is not defined

In [6]:
gdf['matches'].sample(1).values

NameError: name 'gdf' is not defined

In [7]:
sample =  gdf.sample(1)

results = sample.matches.values

print(results[0])

NameError: name 'gdf' is not defined

In [9]:
from datetime import datetime
import json
from pathlib import Path
import requests
from eotdl.tools import bbox_from_centroid
from eotdl.access import download_sentinel_imagery
from dataset import download_images, get_closest_match
from download_images import download_sat_image, download_sentinel_image
import shutil
import os

WIDTH = 38
HEIGHT = 38

shutil.rmtree('sample')
os.makedirs('sample', exist_ok=True)


# starting from HR data (Satellogic), find available S1/S2 that matches in time/area
# take all matches, remove ones with >= 10% cloud cover,

print("collecting...")

# in short, satellogic segment downloads from url using metadata, and saves in chunks to local sample dir
# the sentinal 2 segment downloads image from local path (/fastdata/Satellogic/data/),

# sat (hr) image is huge. sample var is the details of the sat image, including its metadata and where to download it.
# we download the sent2 image of a small box.

# satellogic (HR DATA)

"""
# satellogic (HR DATA)

# get path to metadata file
json_path = sample.json_path.iloc[0]  #
json_path = json_path.replace('data/', '/fastdata/Satellogic/data/')  #
with open(json_path, 'r') as f:
	metadata = json.load(f)

# get download link from metadata
url = metadata['assets']['analytic']['href']

# make path where the sat (hr) image will be saved. download, and then write in chunks to that location.
output_path = Path("sample/satellogic") / url.split('/')[-1]
output_path.parent.mkdir(parents=True, exist_ok=True)
response = requests.get(url, stream=True)
response.raise_for_status()
with open(output_path, 'wb') as f:
	for chunk in response.iter_content(chunk_size=8192):
		f.write(chunk)
"""

print("downloading sat (hr) image...")
date = sample.date.iloc[0]
closest_match = get_closest_match(results, date)  # get
json_path = sample.json_path.iloc[0]  #
centroid = sample.geometry.iloc[0].centroid  # get centroid/center point of sat image.

download_sat_image(json_path, output_path="sample/satellogic")


# sentinel data
custom_bbox = bbox_from_centroid(x=centroid.y, y=centroid.x, pixel_size=10, width=WIDTH, height=HEIGHT)

print("downloading S1...")
sent1_name = json_path.split('/')[-1].replace('_metadata.json', '_S1GRD')
download_sentinel_imagery(
	output="sample/sentinel1",
	time_interval=closest_match["properties"]["datetime"],
	bounding_box=custom_bbox,
	collection_id="sentinel-1-grd",
	name=sent1_name
)
# download_sentinel_image(closest_match,
# 						name=sent1_name,
# 						centroid=centroid,
# 						output_path=dst_path_sentinel1,
# 						collection_id="sentinal-1-grd"
# 						)

print("downloading S2...")
sent2_name = json_path.split('/')[-1].replace('_metadata.json', '_S2L2A')

# download sentinal 2
download_sentinel_imagery(output="sample/sentinel2",
						  time_interval=closest_match["properties"]["datetime"],
						  bounding_box=custom_bbox,
						  collection_id="sentinal-2-l2a",
						  name=sent2_name
						  )

    

ModuleNotFoundError: No module named 'sentinelhub'

In [25]:
import matplotlib.pyplot as plt
import rasterio as rio

fig, axs = plt.subplots(1, 2, figsize=(5, 3))
axs[0].imshow((rio.open(output_path).read()[:3,...].transpose(1, 2, 0) / 3000).clip(0, 1))
axs[0].axis('off')
axs[1].imshow((rio.open(dst_path_sentinel).read()[(3,2,1),...].transpose(1, 2, 0) / 3000).clip(0, 1))
axs[1].axis('off')
plt.show()

ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']

In [29]:
import matplotlib.pyplot as plt


# plot histogram with number of images per zone
fig = plt.figure(figsize=(15, 3))
gdf['zone'].value_counts().plot(kind='bar', ax=fig.gca(), grid=True)
plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
len(gdf['zone'].unique()), len(gdf['region'].unique())

In [ ]:
gdf['zone'].value_counts().min()

In [ ]:
# plot histogram with number of images per month (all images are from 2022)
fig = plt.figure(figsize=(15, 3))
gdf['date'].dt.month.value_counts().plot(kind='bar', ax=fig.gca(), grid=True)
plt.show()


In [ ]:
zones = sorted(gdf['zone'].unique())
zone_gdf = gdf[gdf['zone'] == zones[4]]
sample = zone_gdf.sample(5, random_state=2025)
sample

Download the images

In [7]:
!rm -rf /fastdata/Satellogic/data/tifs/satellogic/*
!rm -rf /fastdata/Satellogic/data/tifs/sentinel2/*

/bin/bash: line 1: /usr/bin/rm: Argument list too long
/bin/bash: line 1: /usr/bin/rm: Argument list too long


In [ ]:
from dataset import find_sentinel_matches

# now supports S2 + S1
matches = []
for row, item in sample.iterrows():
	print(item.json_path)
	sent1_matches = find_sentinel_matches(item.date, item.geometry.bounds, item.json_path, item.centroid, collection_id="sentinal-1-grd")
	sent2_matches = find_sentinel_matches(item.date, item.geometry.bounds, item.json_path, item.centroid, collection_id="sentinal-2-l2a")

	matches.append(sent1_matches + sent2_matches)

In [ ]:
matches

In [ ]:
import rasterio as rio

for match in matches:
	if match is None: continue
	s2_image, sat_image = match
	fig, axs = plt.subplots(1, 2, figsize=(5, 3))
	axs[0].imshow((rio.open(sat_image).read()[:3,...].transpose(1, 2, 0) / 4000).clip(0, 1))
	axs[0].axis('off')
	axs[1].imshow((rio.open(s2_image).read()[(3,2,1),...].transpose(1, 2, 0) / 4000).clip(0, 1))
	axs[1].axis('off')
	plt.show()

